In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.master("local[1]").appName("RDD").getOrCreate()

In [3]:
spark

In [4]:
orders=spark.sparkContext.textFile(r"C:\Users\user\OneDrive\Desktop\Big-data-trendytech\udemy\RetailDB+SalesData\RetailDB SalesData\Orders\part-00000.txt")

In [5]:
for i in orders.take(5): print(i)

1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE


In [6]:
orders_items=spark.sparkContext.textFile(r"C:\Users\user\OneDrive\Desktop\Big-data-trendytech\udemy\RetailDB+SalesData\RetailDB SalesData\Order_items\part-00000.txt")

In [7]:
for i in orders_items.take(5): print(i)

1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99


In [21]:
#Transformations

In [22]:
# 1.Map
#      Perform row level transformations where one record transforms into another record.
#      Number of records in input is equal to output.

In [24]:
# Project all the Order_ids

In [25]:
order_id=orders.map(lambda x:x.split(',')[0])

In [26]:
for i in order_id.take(5): print(i)

1
2
3
4
5


In [27]:
#Project all the Orders and their status.

In [29]:
order_status=orders.map(lambda x:(x.split(',')[0],x.split(',')[3]))

In [30]:
for i in order_status.take(5):print(i)

('1', 'CLOSED')
('2', 'PENDING_PAYMENT')
('3', 'COMPLETE')
('4', 'CLOSED')
('5', 'COMPLETE')


In [ ]:
#Combine Order id and status with ‘#’

In [32]:
order_status_Comb=orders.map(lambda x:(x.split(',')[0]+'#'+x.split(',')[3]))

In [33]:
for i in order_status_Comb.take(5): print(i)

1#CLOSED
2#PENDING_PAYMENT
3#COMPLETE
4#CLOSED
5#COMPLETE


In [34]:
#Convert the Order date into YYYY/MM/DD Format.

In [39]:
orders_calender=orders.map(lambda x:x.split(',')[1]).map(lambda x:x.split(' ')[0])

In [40]:
for i in orders_calender.take(5): print(i)

2013-07-25
2013-07-25
2013-07-25
2013-07-25
2013-07-25


In [43]:
#Applied user defined function to convert status into lowercase.

In [64]:
def lowerCase(str):
    return str.lower()

In [65]:
order_lower=orders.map(lambda x:lowerCase(x.split(',')[3]))

In [66]:
import string

In [67]:
order_lower.take(5)

['closed', 'pending_payment', 'complete', 'closed', 'complete']

In [68]:
#2.flatMap
   #Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results.
   #Similar to map, but each input item can be mapped to 0 or more output items . Number of records in input is less than or equal to output.

In [ ]:
#Word count in orders file.

In [86]:
orders.flatMap(lambda x:(x.split(','))).map(lambda w:(w,1)).reduceByKey(lambda x,y:x+y).take(5)

[('1', 2),
 ('2013-07-25 00:00:00.0', 143),
 ('11599', 6),
 ('CLOSED', 7556),
 ('2', 5)]

In [ ]:
#Filter

In [87]:
#Print all the orders which are closed or Complete and ordered in the year 2013.

In [99]:
orders.filter(lambda x:(x.split(',')[3] in ["CLOSED","COMPLETE"]) and (x.split(',')[1].split('-')[0]=="2013")).take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE']

In [100]:
#mapValues()

In [101]:
#Only applicable to pair RDDs.
#Apply the function ‘f’ to all values of the same key.
#The difference with map is map operates on the entire record.

In [107]:
rdd_mv = spark.sparkContext.parallelize ((("a", (1,2,3)), ("b", (3,4,5)),("a", (1,2,3,4,5))))

In [108]:
def f(x):
    return len(x)

In [109]:
rdd_mv.mapValues(f).collect()

[('a', 3), ('b', 3), ('a', 5)]

In [114]:
#Join

In [115]:
#Find the subtotal for each ORDER_CUSTOMER_ID.

In [116]:
orders.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [117]:
orders_items.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [119]:
orders1=orders.map(lambda x:(x.split(',')[0],x.split(',')[2]))

In [120]:
orders1.take(5)

[('1', '11599'), ('2', '256'), ('3', '12111'), ('4', '8827'), ('5', '11318')]

In [121]:
orders_items_1=orders_items.map(lambda x:(x.split(',')[1],x.split(',')[4]))

In [123]:
orders_items_1.take(5)

[('1', '299.98'),
 ('2', '199.99'),
 ('2', '250.0'),
 ('2', '129.99'),
 ('4', '49.98')]

In [125]:
full_orders=orders1.join(orders_items_1)

In [126]:
full_orders.take(5)

[('1', ('11599', '299.98')),
 ('4', ('8827', '49.98')),
 ('4', ('8827', '299.95')),
 ('4', ('8827', '150.0')),
 ('4', ('8827', '199.92'))]

In [132]:
subtotal_id_1=full_orders.map(lambda x:(str(x[1][0])+','+str(x[1][1])))

In [133]:
subtotal_id_1.take(5)

['11599,299.98', '8827,49.98', '8827,299.95', '8827,150.0', '8827,199.92']

In [135]:
#Examples using Reduce and Count

In [8]:
orders.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [9]:
# 1. Count the number of orders which are closed.

In [10]:
orders.filter(lambda x:(x.split(',')[3]=='CLOSED')).count()

7556

In [11]:
# 2. Find the total quantity sold for Order ID 1-10.

In [12]:
orders_items.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [23]:
orders_items.filter(lambda x:int(x.split(',')[1])<11).map(lambda x:float(x.split(',')[3])).reduce(lambda x,y:x+y)

62.0

In [24]:
from operator import add

In [25]:
orders_items.filter(lambda x:int(x.split(',')[1])<11).map(lambda x:float(x.split(',')[3])).reduce(add)

62.0

In [26]:
# For a given order 10 find the maximum subtotal out of all orders.

In [30]:
orders_items.filter(lambda x:int(x.split(',')[1])==10).map(lambda x:float(x.split(',')[4])).reduce(max)

199.99

In [31]:
#count the orders in each category

In [33]:
orders.map(lambda x:(x.split(',')[3],1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],False).collect()

[('COMPLETE', 22899),
 ('PENDING_PAYMENT', 15030),
 ('PROCESSING', 8275),
 ('PENDING', 7610),
 ('CLOSED', 7556),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558),
 ('CANCELED', 1428),
 ('PAYMENT_REVIEW', 729)]

In [8]:
# Find the maximum revenue for each order.

In [11]:
orders_items.map(lambda x:(x.split(',')[1],x.split(',')[4])).reduceByKey(lambda x,y:x if x>y else y).take(5)

[('1', '299.98'),
 ('2', '250.0'),
 ('4', '49.98'),
 ('5', '99.96'),
 ('7', '79.95')]

In [12]:
#using aggregateByKey()

In [13]:
#Find the maximum revenue for each Order.

In [14]:
ordItems=spark.sparkContext.parallelize([(2,"Joseph",200), (2,"Jimmy",250), (2,"Tina",130), (4,"Jimmy",50), (4,"Tina",300),
(4,"Joseph",150), (4,"Ram",200), (7,"Tina",200), (7,"Joseph",300), (7,"Jimmy",80)],2)

In [15]:
ordItems.collect()

[(2, 'Joseph', 200),
 (2, 'Jimmy', 250),
 (2, 'Tina', 130),
 (4, 'Jimmy', 50),
 (4, 'Tina', 300),
 (4, 'Joseph', 150),
 (4, 'Ram', 200),
 (7, 'Tina', 200),
 (7, 'Joseph', 300),
 (7, 'Jimmy', 80)]

In [16]:
ordPair=ordItems.map(lambda x:(x[0],(x[1],x[2])))

In [17]:
ordPair.take(3)

[(2, ('Joseph', 200)), (2, ('Jimmy', 250)), (2, ('Tina', 130))]

In [18]:
#Initialize Accumulator
# Zero Value: Zero value in our case will be 0 as we are finding Maximum Marks
zero_val=0

In [19]:
#Define Sequence Operation
# Sequence operation : Finding Maximum revenue from each partition

def seq_op(accumulator,element):
    if(accumulator>element[1]):
        return accumulator
    else:
        return element[1]

In [20]:
#Define Combiner Operation
#Combiner Operation : Finding Maximum revenue from all partitions

In [21]:
def comb_op(accumulator1,accumulator2):
    if(accumulator1>accumulator2):
        return accumulator1
    else:
        return accumulator2

In [25]:
aggr_ordItems=ordPair.aggregateByKey(zero_val,seq_op,comb_op)

In [26]:
for i in aggr_ordItems.collect(): print(i)

(2, 250)
(4, 300)
(7, 300)


In [27]:
#using countByKey()

In [8]:
orders.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [9]:
orders_map=orders.map(lambda x:x.split(',')[3],1)

In [17]:
orders_map.take(5)

['CLOSED', 'PENDING_PAYMENT', 'COMPLETE', 'CLOSED', 'COMPLETE']

In [10]:
orders_status_count=orders_map.countByKey()

In [11]:
type(orders_status_count)

collections.defaultdict

In [12]:
# as the type is collections dictionary it will not work with collect() and take() so use items(),values(),keys()

In [15]:
for i in orders_status_count.values(): print(i)

31883
31644
3798
1558


In [16]:
for i in orders_status_count.items(): print(i)

('C', 31883)
('P', 31644)
('O', 3798)
('S', 1558)


In [18]:
#using sortByKey()

In [19]:
# 1 Sort orders using customer id

In [20]:
orders.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [27]:
data_sort=orders.map(lambda x:(int(x.split(',')[2]),x))

In [29]:
data_sort.take(1)

[(11599, '1,2013-07-25 00:00:00.0,11599,CLOSED')]

In [30]:
sorted_value=data_sort.sortByKey()

In [31]:
for i in sorted_value.take(5):print(i)

(1, '22945,2013-12-13 00:00:00.0,1,COMPLETE')
(2, '15192,2013-10-29 00:00:00.0,2,PENDING_PAYMENT')
(2, '33865,2014-02-18 00:00:00.0,2,COMPLETE')
(2, '57963,2013-08-02 00:00:00.0,2,ON_HOLD')
(2, '67863,2013-11-30 00:00:00.0,2,COMPLETE')


In [32]:
# 2 Sort orders using customer and status.

In [34]:
data_sort1=orders.map(lambda x:(int(x.split(',')[2]),x.split(',')[3],x))

In [35]:
data_sort1.take(1)

[(11599, 'CLOSED', '1,2013-07-25 00:00:00.0,11599,CLOSED')]

In [36]:
sorted_value1=data_sort1.sortByKey()

In [37]:
for i in sorted_value1.take(5):print(i)

(1, 'COMPLETE', '22945,2013-12-13 00:00:00.0,1,COMPLETE')
(2, 'PENDING_PAYMENT', '15192,2013-10-29 00:00:00.0,2,PENDING_PAYMENT')
(2, 'COMPLETE', '33865,2014-02-18 00:00:00.0,2,COMPLETE')
(2, 'ON_HOLD', '57963,2013-08-02 00:00:00.0,2,ON_HOLD')
(2, 'COMPLETE', '67863,2013-11-30 00:00:00.0,2,COMPLETE')


In [38]:
#union()

In [39]:
orders.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [40]:
#Number of customers placed order in July or Aug Month.

In [45]:
july_orders=orders.filter(lambda x:(x.split(',')[1].split('-')[1])=='07').map(lambda x:(x.split(',')[2]))

In [46]:
Aug_orders=orders.filter(lambda x:(x.split(',')[1].split('-')[1])=='08').map(lambda x:(x.split(',')[2]))

In [47]:
july_orders.union(Aug_orders).distinct().count()

7633

In [48]:
#intersection()

In [50]:
july_orders.intersection(Aug_orders).count()

1759

In [51]:
#subtract()

In [52]:
july_orders.subtract(Aug_orders).count()

3836